In [ ]:
# !pip install transformers

In [ ]:
# import os
# # import Google Drive 套件
# from google.colab import drive
# # 將自己的雲端硬碟掛載上去
# drive.mount('/content/gdrive')

# os.chdir('./gdrive/MyDrive/成大/justLearn/Jigsaw Rate Severity of Toxic Comments/')      # 檔案目錄

In [1]:
from transformers import AutoTokenizer, AutoModel
import pickle
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.linear_model import Ridge
import joblib
from tqdm import tqdm

In [2]:
# load test data
path = './data/testData/comments_to_score.csv'
data = pd.read_csv(path)

In [3]:
# 留下前510字的 function
def stringProcess(data):
    input_ids, token_type_ids, attention_mask = data['input_ids'], data['token_type_ids'], data['attention_mask']
    data['input_ids'] = data['input_ids'][:, :512]
    data['input_ids'][:, -1] = 102
    data['token_type_ids'] = data['token_type_ids'][:, :512]
    data['attention_mask'] = data['attention_mask'][:, :512]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [4]:
# 載入並宣告bert模型
# tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
# bert_model = AutoModel.from_pretrained("bert-base-cased").to(device)

tokenizer = AutoTokenizer.from_pretrained("./model/tokenizer/")
# bert_model = torch.load('./model/bert_model.pt').to(device)

# ridge
ridge_model = joblib.load('./model/ridge_model.joblib')

C:\Users\Beefnoodle\anaconda3\envs\datascience\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator Ridge from version 1.0.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [5]:
bert_model = BertModel()

TypeError: __init__() missing 1 required positional argument: 'config'

In [27]:
savePath = './result/ridge_result.csv'
n = data.shape[0]
score = []
for i in tqdm(range(n)):
    d = data.iloc[i, :]
    comment_id = d['comment_id']
    text = d['text'].replace('\n', ' ')
    with torch.no_grad():
        inputs = tokenizer(text, return_tensors="pt").to(device)
        stringProcess(inputs)
        token = bert_model(**inputs)[1].cpu().numpy()
    s = ridge_model.predict(token)
    score.append(s[0])
#     if i ==10:
#         break
        
ids = data['comment_id']
score = score
outputs_data = pd.DataFrame({'comment_id':ids, 'score':score})
outputs_data.to_csv(savePath, index=False)

  0%|          | 10/7537 [00:01<19:05,  6.57it/s]
